In [2]:
import pandas as pd
import numpy as np
import mysql.connector

# Connect to MySQL database
connection = mysql.connector.connect(
    host="localHost",
    user="root",
    password="Sonji@2534",
    database="Hackathon"
)

# Define the SQL query to fetch data
query = "SELECT * FROM newdatasheet"

# Fetch data into a DataFrame
data = pd.read_sql(query, connection)

# Close the database connection
connection.close()

# Adjust age probability values to increase importance
age_probabilities = {
    '0-20': 0.1,
    '21-30': 0.3,
    '31-40': 0.5,
    '41-50': 0.6,
    '51-60': 0.5,
    '61-70': 0.3,
    '71+': 0.1
}

# Clean up profession labels and convert to lowercase
data['Profession'] = data['Profession'].str.strip().str.lower()

# Update profession values dictionary
profession_values = {
    "labourer": 1,
    "breozgar": 2,
    "driver": 3,
    "farmer": 4,
    "housewife": 5,
    "businessman": 6,
    "student": 7,
    "others pi specify": 8,
    "ceo": 9,
    "para medical staff": 10,
    "carpenter": 11,
    "politicians": 12,
    "tailor": 13,
    "plumber": 14,
    "driver-autorickshaw": 15,
    "electrician": 16
}

# Adjust district crime rates to increase importance
district_crime_rates = {
    'Bagalkot': 100,
    'Bengaluru City': 200,
    'Bidar': 150
}

# Considering gender distribution
total_accused = len(data)
male_count = data['Sex'].value_counts().get('MALE', 0)
female_count = data['Sex'].value_counts().get('FEMALE', 0)
male_probability = male_count / total_accused
female_probability = female_count / total_accused

# Define the additional contribution for gender
male_contribution = 0.1  # Adjust as needed
female_contribution = 0.05  # Adjust as needed

# Group age into predefined age ranges
age_bins = [0, 20, 30, 40, 50, 60, 70, np.inf]
age_labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71+']
data['AgeGroup'] = pd.cut(data['ï»¿age'], bins=age_bins, labels=age_labels, right=False)

# Combine probabilities
data['CriminalityProbability'] = (data['AgeGroup'].map(age_probabilities) + 
                                  data['Profession'].map(profession_values) + 
                                  data['District_Name'].map(district_crime_rates) +
                                  np.where(data['Sex'] == 'MALE', male_contribution, female_contribution))

# Generate predictions
threshold_probability = 125  # Adjust threshold as needed
data['Prediction'] = data['CriminalityProbability'] >= threshold_probability

# Output results in terms of labels based on threshold probability
def label_probability(probability):
    if probability >= threshold_probability:
        return "Most likely"
    else:
        return "Not likely"

# Apply the function to the CriminalityProbability column
data['Prediction'] = data['CriminalityProbability'].apply(label_probability)

# Output the CriminalityProbability and Prediction for each data entry
print(data[['CriminalityProbability', 'Prediction']])


     CriminalityProbability   Prediction
0                    101.40   Not likely
1                    102.40   Not likely
2                    102.40   Not likely
3                    102.40   Not likely
4                    102.40   Not likely
..                      ...          ...
131                  152.55  Most likely
132                  152.60  Most likely
133                  152.20  Most likely
134                  152.55  Most likely
135                  152.40  Most likely

[136 rows x 2 columns]


C:\Users\sonji\AppData\Local\Temp\ipykernel_1644\2012491693.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, connection)
